## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
!pip install python-docx python-pptx

In [2]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [3]:
import os
from langchain.schema import Document
import docx
import pptx
import json

def load_documents(path="knowledge_base"):
    documents = []
    for root, _, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            content = ""
            try:
                if file.endswith(".md"):
                    with open(file_path, "r", encoding="utf-8") as f:
                        content = f.read()
                elif file.endswith(".txt"):
                    with open(file_path, "r", encoding="utf-8") as f:
                        content = f.read()
                elif file.endswith(".docx"):
                    doc = docx.Document(file_path)
                    content = "\n".join([para.text for para in doc.paragraphs])
                elif file.endswith(".pptx"):
                    pres = pptx.Presentation(file_path)
                    for slide in pres.slides:
                        for shape in slide.shapes:
                            if hasattr(shape, "text"):
                                content += shape.text + "\n"
                elif file.endswith(".json"):
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                        content = json.dumps(data, indent=2)
                
                if content:
                    # Get the parent directory name as the doc_type
                    doc_type = os.path.basename(root)
                    documents.append(Document(page_content=content, metadata={"source": file_path, "doc_type": doc_type}))
            except Exception as e:
                print(f"Error loading file {file_path}: {e}")
    return documents

In [4]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [6]:
documents = load_documents()
print(f"Loaded {len(documents)} documents.")

Loaded 4 documents.


In [7]:
# Filter out documents with no content before splitting
valid_documents = [doc for doc in documents if doc.page_content and doc.page_content.strip()]
print(f"Found {len(documents)} documents, with {len(valid_documents)} being valid after filtering.")

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(valid_documents)

Found 4 documents, with 4 being valid after filtering.


In [8]:
len(chunks)

4

In [9]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: drive, slack


## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote

In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [10]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OpenAIEmbeddings()
collection_name = "insurellm_docs" # Use a consistent collection name

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# This notebook is designed to build the DB from scratch.
# The watchdog script (`untitled.py`) is designed to keep it updated.
# For the live-update to work, we should not delete the collection every time.
# We will connect to it, and if it's empty, we will populate it.

# Connect to the vectorstore
vectorstore = Chroma(
    persist_directory=db_name, 
    embedding_function=embeddings,
    collection_name=collection_name
)

# If the database is empty, populate it from the documents loaded earlier.
if vectorstore._collection.count() == 0:
    print("Database is empty. Populating with initial documents...")
    vectorstore.add_documents(chunks)
    print(f"Vectorstore populated with {vectorstore._collection.count()} documents.")
else:
    print(f"Connected to existing vectorstore with {vectorstore._collection.count()} documents.")

/tmp/ipykernel_61083/4068487649.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Connected to existing vectorstore with 55 documents.


In [11]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [12]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# Create a ChatOpenAI model
llm = ChatOpenAI(model_name=MODEL, temperature=0)

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [13]:
!pip install gradio watchdog

In [14]:
import gradio as gr
import time
import threading
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Global flag to signal when the knowledge base has been updated
update_notification = ""

def load_single_document(file_path):
    """Loads a single document from a file path."""
    content = ""
    doc_type = os.path.basename(os.path.dirname(file_path))
    try:
        if file_path.endswith((".md", ".txt")):
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
        elif file_path.endswith(".docx"):
            import docx
            doc = docx.Document(file_path)
            content = "\n".join([p.text for p in doc.paragraphs])
        elif file_path.endswith(".pptx"):
            import pptx
            pres = pptx.Presentation(file_path)
            content = "\n".join(
                shape.text for slide in pres.slides for shape in slide.shapes if hasattr(shape, "text")
            )
        elif file_path.endswith(".json"):
            import json
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                content = json.dumps(data, indent=2)
        
        if content:
            return [Document(page_content=content, metadata={"source": file_path, "doc_type": doc_type})]
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
    return None

class KnowledgeBaseWatcher(FileSystemEventHandler):
    def __init__(self, vectorstore, text_splitter):
        self.vectorstore = vectorstore
        self.text_splitter = text_splitter

    def on_modified(self, event):
        if not event.is_directory:
            self.update_vectorstore(event.src_path)

    def on_created(self, event):
        if not event.is_directory:
            self.update_vectorstore(event.src_path)

    def update_vectorstore(self, file_path):
        global update_notification
        print(f"Detected change in {file_path}. Updating vector store.")
        
        # Delete existing documents from the vector store that match the source file
        existing_docs = self.vectorstore.get(where={'source': file_path})
        if existing_docs['ids']:
            self.vectorstore.delete(ids=existing_docs['ids'])
            print(f"Deleted {len(existing_docs['ids'])} old chunks for {file_path}")

        # Load the new/modified document and add it to the vector store
        docs = load_single_document(file_path)
        if docs:
            chunks = self.text_splitter.split_documents(docs)
            self.vectorstore.add_documents(chunks)
            print(f"Added {len(chunks)} new chunks for {file_path}")
            update_notification = f"Knowledge base updated with changes from: {os.path.basename(file_path)}"

def start_watcher(path, vectorstore, text_splitter):
    event_handler = KnowledgeBaseWatcher(vectorstore, text_splitter)
    observer = Observer()
    observer.schedule(event_handler, path, recursive=True)
    observer.start()
    print(f"Watching for changes in {path}")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Start the file watcher in a background thread
watcher_thread = threading.Thread(
    target=start_watcher,
    args=("knowledge_base", vectorstore, text_splitter),
    daemon=True
)
watcher_thread.start()

def chat(message, history):
    global update_notification
    
    # Check if there's an update notification to display
    if update_notification:
        notification_message = f"✨ *Knowledge base was recently updated. The answer below may reflect new information.*\n\n---\n"
        update_notification = "" # Reset after displaying
    else:
        notification_message = ""

    response = qa_chain.invoke({"query": message})
    return notification_message + response["result"]

iface = gr.ChatInterface(fn=chat, title="Knowledge Base Chat", chatbot=gr.Chatbot(height=600))
iface.launch()

/home/variable/One_Stop_Ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_61083/4014041867.py:105: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  iface = gr.ChatInterface(fn=chat, title="Knowledge Base Chat", chatbot=gr.Chatbot(height=600))
/home/variable/One_Stop_Ai/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Watching for changes in knowledge_base
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
import docx

# Create a new Document
doc = docx.Document()

# Add a heading
doc.add_heading('AI Team Members', level=1)

# Add team member names
team_members = [
    "Alice - Lead AI Scientist",
    "Bob - Machine Learning Engineer",
    "Charlie - Data Scientist",
    "David - AI Ethicist",
    "Eve - NLP Specialist"
]

for member in team_members:
    doc.add_paragraph(member, style='List Bullet')

# Save the document
file_path = "/home/thunder/projects/one_stop/knowledge_base/drive/ai_team_members.docx"
doc.save(file_path)

print(f"Created dummy docx file at: {file_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/home/thunder/projects/one_stop/knowledge_base/drive/ai_team_members.docx'

Created a chunk of size 1343, which is longer than the specified 1000
Created a chunk of size 1246, which is longer than the specified 1000
Created a chunk of size 1812, which is longer than the specified 1000
Created a chunk of size 4615, which is longer than the specified 1000
Created a chunk of size 24443, which is longer than the specified 1000


Detected change in knowledge_base/slack/20250829_235312_CREATE DEFINER user TRIGGER.txt. Updating vector store.


Created a chunk of size 1343, which is longer than the specified 1000
Created a chunk of size 1246, which is longer than the specified 1000
Created a chunk of size 1812, which is longer than the specified 1000
Created a chunk of size 4615, which is longer than the specified 1000
Created a chunk of size 24443, which is longer than the specified 1000


Added 25 new chunks for knowledge_base/slack/20250829_235312_CREATE DEFINER user TRIGGER.txt
Detected change in knowledge_base/slack/20250829_235312_CREATE DEFINER user TRIGGER.txt. Updating vector store.
Deleted 25 old chunks for knowledge_base/slack/20250829_235312_CREATE DEFINER user TRIGGER.txt
Added 25 new chunks for knowledge_base/slack/20250829_235312_CREATE DEFINER user TRIGGER.txt
